# Euro Millions

By ***Rok Medves***

Remarks on dataset can be found in ./datasets/README.md


## Loading the dataset

In [159]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

# Load in the data -- could be automated to just pull the data from the webpage
euromillions_draws   = pd.read_csv('./datasets/euro-millions-draws.csv',   index_col=0)
euromillions_winners = pd.read_csv('./datasets/euro-millions-winners.csv', index_col=0)
euromillions_sales   = pd.read_csv('./datasets/euro-millions-sales.csv', usecols=[4,])

euromillions_sales = euromillions_sales.set_index(euromillions_winners.index)

# clean column names
for dataset in (euromillions_draws, euromillions_winners, euromillions_sales):
    dataset.columns = [ col.strip() for col in dataset.columns]

# only temporary:
# 
# On sep 2016 the number of winning numbers changed for the last time (see ./datasets/README.md)
# This flag selects only data after that date
select_only_pre_sep_2016_data = True


## Dataset Cleaning and Processing

In [160]:
# ---------------------------- handing NaN values in sales dataset -----------------------------------------

# there is a N/A in the sales dataset 
# find it and set the sales value to the expected total.
# This needs to be done manually because of spaces 
# invalidating the pd.isnull() & pd.isna() functions
#
# the expected number of sales should roughly be 13 * the
# total number of winners
na_loc = euromillions_sales.Sales.str.strip() == 'N/A'
na_ind = euromillions_sales[na_loc].index[0]

# Replace the `N/A` with a values reflecting the sales/winners mean. 
# it's quick and dirty, but it gets the ratio right
euromillions_sales.loc[na_ind] = euromillions_sales.Sales.drop(index = na_ind).astype(float).mean()/euromillions_winners.Total.mean() * euromillions_winners.Total.loc[na_ind]
euromillions_sales = euromillions_sales.astype(float)

# ---------------------------- Filling in missing values for number of winners -----------------------------------------
# When the category with 2 matching numbers is introduced, the dataset gains a column. 
# The following code amends the missing values by setting the number of winners in the group `2` to 0
# Admittedly, this could be done better!
# TODO: Find a better way of filling in missing values without messing up the predictive power of the model

#Splitting the dataset into a NaN region and a non-Nan region
winners_nan, winners_nonan = euromillions_winners[pd.isna(euromillions_winners.Total) == True], euromillions_winners[pd.isna(euromillions_winners.Total) == False],

# Reindexing things
winners_nan.loc[:, '2'], winners_nan.loc[:, 'Total'] = winners_nan.loc[:, 'Total'].copy(), winners_nan.loc[:, '2'].copy()

# Replacing all NaN with 0
winners_nan = winners_nan.fillna(0)

# Combining everything
euromillions_winners = pd.concat([winners_nonan, winners_nan])

# Now we can actually combine the whole dataset 
euromillions = pd.concat([euromillions_draws, euromillions_winners, euromillions_sales], axis = 1)

# ---------------------------- Validate the cleaning procedure -----------------------------------------

# Validate the clean
summed_winnings = euromillions.loc[:, '5+2':'2'].apply(np.sum, axis = 'columns')
total_validation = pd.concat([summed_winnings, euromillions.Total], axis = 1).rename(columns={0: 'true', 'Total' : 'dataset'})

def validate_cleaning(df : pd.DataFrame):
    """
    Validates the cleaning procedure by
    checking whether the total number of
    winners is consistent accross the dataset

    Parameters
    ----------
    df : pandas.Dataframe
        The dataframe to validate the cleaning of

    Returns
    -------
    bool
        Whether the dataset is lean or not
    """

    clean = True

    df_notnan = df[pd.notna(df.dataset)]
    df_diff    = df_notnan.apply(lambda df: df.dataset - df.true, axis = 'columns')
    if df_diff.sum() != 0.0:
        print("The original total columns were wrong!")
        print(df_notnan[df_diff != 0])

        clean = False

    return clean

if validate_cleaning(total_validation): print("Dataset cleaned accordingly")

# trim extra values; flag set at the top, when loading the data
def trim_by_date(df, do_select = False) -> pd.DataFrame:
    """ 
    Trims the dataset df with respect to the date.
    Note that this date is hard-coded to match when the 
    euromillions changed the range of their lucky numbers from 11 to 12

    Parameters
    ----------
    df : pandas.Dataframe
        The dataframe we trim
    do_select : bool
        whether or not to do the selection
    
    Returns
    -------
    pandas.Dataframe
        The trimmed array

    """
    min_index = 0
    if do_select:
        date_mask = (euromillions.DD == 27) & (euromillions.MMM == 'Sep') & (euromillions.YYYY == 2016)
        min_index = euromillions[date_mask].index[0]
    
    print("additionally removing {0} entries from dataset".format(min_index))
    return df.loc[df.index >= min_index]

euromillions = trim_by_date(euromillions, select_only_pre_sep_2016_data)

# euromillions  = euromillions.drop(columns = ['Day', 'DD', 'MMM', 'YYYY', 'Wins'])


Dataset cleaned accordingly
additionally removing 941 entries from dataset


## Feature Engineering

I speculate that people like to bet on numbers that are either
- dates (birthdays, anniversaries, etc.)
- 'nice' numbers, such as 3, 7, 13

To leverage this, new features are engineered below 

In [161]:
def is_date(c : pd.Series) -> bool:
    """ 
    Checks whether the row c contains
    a valid date. E.g.
    5. 10.  -- > True
    if no such dates are found, it returns false

    Parameters
    ----------
    c : pandas.Series
        the rows that will be checked
    
    Returns
    -------
    bool
        the truth value of whether the row 
        contains a valid date
    """
    
    day = False
    month = False
    for lab, i in c.iteritems():
        if 'N' not in lab: continue

        if not month:
            if i <= 12: month = True
        elif not day:
            if i <= 31: day = True
        else:
            break
    return day and month

def is_this_year(c : pd.Series) -> bool:
    """ 
    Checks whether the row c contains
    the same year that the draw happend

    Parameters
    ----------
    c : pandas.Series
        the rows that will be checked
    
    Returns
    -------
    bool
        the truth value of whether the row 
        contains the current year
    """
    first = 20
    second = c['YYYY']-2000

    f, s = False, False
    for lab, i in c.iteritems():
        if 'N' not in lab: continue

        if   i == first:  f = True
        elif i == second: s = True
    
    return f and s

def is_post_2000(c : pd.Series) -> bool:
    """ 
    Checks whether the row c contains
    the a valid year past the year 2000

    Parameters
    ----------
    c : pandas.Series
        the rows that will be checked
    
    Returns
    -------
    bool
        the truth value of whether the row 
        contains a year after 2000
    """
    first = 20
    second = c['YYYY']-2000

    f, s = False, False
    for lab, i in c.iteritems():
        if 'N' not in lab: continue

        if   i == 20:  f = True
        elif i <= second: s = True
    
    return f and s

N_numbers = euromillions.loc[:,'YYYY':'N5']
euromillions["is_date"]      = N_numbers.apply(is_date, axis = 1)
euromillions["is_post_2000"] = N_numbers.apply(is_post_2000, axis = 1)
euromillions["is_this_year"] = N_numbers.apply(is_this_year, axis = 1)

In [162]:
euromillions

,Day,DD,MMM,YYYY,N1,N2,N3,N4,N5,L1,...,3+1,3,1+2,2+1,2,Total,Sales,is_date,is_post_2000,is_this_year
No.,,,,,,,,,,,,,,,,,,,,,
1611,Tue,28,Feb,2023,12,29,33,37,47,5,...,6089,13624,22900,85714,197798.0,331160.0,5842047.0,True,False,False
1610,Fri,24,Feb,2023,7,23,34,42,48,1,...,11687,25359,42100,167578,370796.0,626658.0,60482734.0,True,False,False
1609,Tue,21,Feb,2023,21,22,34,41,49,2,...,7402,16155,37953,115458,246870.0,431066.0,46454399.0,False,False,False
1608,Fri,17,Feb,2023,8,23,27,42,45,1,...,10838,25228,39150,153744,358568.0,596354.0,53000419.0,True,False,False
1607,Tue,14,Feb,2023,24,26,38,43,46,2,...,6713,14292,24843,101686,214041.0,366780.0,40015726.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,Tue,11,Oct,2016,5,12,20,24,37,4,...,17862,41337,60710,249292,571509.0,954881.0,84831239.0,True,True,False
944,Fri,7,Oct,2016,2,12,20,27,30,9,...,20032,47594,63498,269228,629346.0,1044604.0,98334429.0,True,True,False
943,Tue,4,Oct,2016,4,7,14,34,38,4,...,13524,28529,45972,186159,415184.0,699751.0,70457205.0,True,False,False
